### 01_cassandra_setup.ipynb
### Purpose: Set up Cassandra keyspace and table, and test Spark–Cassandra connection

In [1]:
from pyspark.sql import SparkSession
from cassandra.cluster import Cluster
from datetime import datetime

In [7]:
# Start Spark session with Cassandra connector
spark = SparkSession.builder \
    .appName("CassandraSetup") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .config("spark.cassandra.connection.keepAliveMS", "60000") \
    .getOrCreate()

print("✅ SparkSession started with Cassandra integration")

✅ SparkSession started with Cassandra integration


25/10/13 11:11:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
# Connect directly to Cassandra
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Create keyspace (only if it doesn't already exist)
session.execute("""
DROP TABLE IF EXISTS elhub_data.production_raw
""")

session.execute("""
CREATE TABLE elhub_data.production_raw (
    priceArea TEXT,
    startTime TIMESTAMP,
    endTime TIMESTAMP,
    productionGroup TEXT,
    quantityKwh DOUBLE,
    meteringgridarea TEXT,
    lastUpdatedTime TIMESTAMP,
    PRIMARY KEY (priceArea, startTime, productionGroup)
)
""")

print("✅ Table 'production_raw' recreated to match Elhub API schema")


✅ Table 'production_raw' recreated to match Elhub API schema


In [4]:
# Sample data to test write
data = [
    ("NO1", datetime.utcnow(), "Hydro", "A04", 123.45, "PT1H", "RES001", datetime.utcnow())
]
columns = [
    "meteringgridarea", "time_start", "productiontype", "businesstype",
    "quantity", "resolution", "resourceid", "time_end"
]

df = spark.createDataFrame(data, columns)

# Write to Cassandra
df.write.format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .options(table="production_raw", keyspace="elhub_data") \
    .save()

print("✅ Test row written to Cassandra successfully")

/var/folders/db/2wck1jtn25v9rddfx0kw90qr0000gn/T/ipykernel_58493/722186836.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ("NO1", datetime.utcnow(), "Hydro", "A04", 123.45, "PT1H", "RES001", datetime.utcnow())


✅ Test row written to Cassandra successfully


In [5]:
df_read = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="production_raw", keyspace="elhub_data") \
    .load()

df_read.show()
print(f"✅ Read {df_read.count()} rows from Cassandra")

+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+
|meteringgridarea|          time_start|productiontype|businesstype|quantity|resolution|resourceid|            time_end|
+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+
|             NO1|2025-10-13 08:49:...|         Hydro|         A04|  123.45|      PT1H|    RES001|2025-10-13 08:49:...|
+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+

✅ Read 1 rows from Cassandra


### Delete Rows in the Table and Keeps Columns and Keys

In [6]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect('elhub_data')
session.execute("TRUNCATE production_raw;")

df_read.show()
print("✅ Table elhub_data.production_raw has been cleared.")

+----------------+----------+--------------+------------+--------+----------+----------+--------+
|meteringgridarea|time_start|productiontype|businesstype|quantity|resolution|resourceid|time_end|
+----------------+----------+--------------+------------+--------+----------+----------+--------+
+----------------+----------+--------------+------------+--------+----------+----------+--------+

✅ Table elhub_data.production_raw has been cleared.
